In [273]:
import pandas as pd
import numpy as np
import matplotlib as plt
import warnings
import statistics
from google.cloud import bigquery as bq
%matplotlib inline

# Gain access to the account where the data is located
client = bq.Client.from_service_account_json("C:/Users/Andrew/Documents/Data Science Masters/Applied Statistics 6372/Projects/Project 1/data/ml-jth-d220ef33563e.json")
# Suppress warnings because they're annoying.
warnings.simplefilter(action="ignore")

## Read in the data

In [274]:
bombing = pd.read_csv("WW2_Bombing_Data.csv")
stations = pd.read_csv("WeatherStationsGermany.csv")

#### Merging data sets
We need to determine which weather stations were in the vicinity of a given bombing mission to extract what the weather was like at that station.  To do this, we'll consider that latitude and longitude at which the attack took place, and then create a region around that location representing what the weather was like there.

At first glance, it makes sense to consider the range that the aircraft can see to the horizon.  The formula for distance in miles is 1.22 times the square root of the height (in feet) of the aircraft.

\begin{equation}
Distance_{Horizon}(Miles) = 1.22\times\sqrt{Height(Feet)}
\end{equation}

WW2 bombing aircraft generally flew at an altitude of 10,000 feet when delivering their payload.  Using 10,000 feet for height, we see that aircraft can see approximately 122 miles to the horizon in all directions.

Each degree of lattitude is approximately 69 miles.  Therefore it's safe to say that the weather will likely be unchanged in a 2 degree latitude radius (~140 miles).

In [275]:
# Define a function that will list all station IDs within a 1 degree wide box centered around the attack
def FindStations(Latitude, Longitude):
    nearby_stations = stations[(Latitude+2 >= stations["Latitude"]) & (Latitude-2 <= stations["Latitude"]) &
                              (Longitude+2 >= stations["Longitude"]) & (Longitude-2 <= stations["Longitude"])]
    return(list(nearby_stations["StationID"]))

In [276]:
bombing_stations = bombing[bombing["DefCountry"] == "Germany"]  # Only look at bombings in Germany
bombing_stations.drop(["Details"], axis=1, inplace = True)  # We don't need the Details column, so drop it.
bombing_stations["StationIDs"] = ""  # This is just a place holder so the next line will work
# Iterate through each row, and pass the lat and long into the FindStations function
for index, row in bombing_stations.iterrows():
    bombing_stations.at[index, "StationIDs"]= FindStations(float(row["Latitude"]), float(row["Longitude"]))

This produces a new dataframe that looks like the one presented below.  The StationIDs are presented as a lit of IDs of the weather stations within range of the given attack.

In [277]:
bombing_stations.head()

,Unnamed: 0,Success,Month,Day,Year,Country,DefCity,DefCountry,Latitude,Longitude,StationIDs
3,3,0,9,3,1939,United Kingdom,Wilhelmshaven,Germany,53.53234,8.106872,"[44, 78, 102, 172, 185, 243, 294, 326, 327, 34..."
4,4,0,9,3,1939,United Kingdom,Wilhelmshaven,Germany,53.53234,8.106872,"[44, 78, 102, 172, 185, 243, 294, 326, 327, 34..."
5,5,0,9,4,1939,United Kingdom,Wilhelmshaven,Germany,53.53234,8.106872,"[44, 78, 102, 172, 185, 243, 294, 326, 327, 34..."
6,6,0,9,4,1939,United Kingdom,Wilhelmshaven,Germany,53.53234,8.106872,"[44, 78, 102, 172, 185, 243, 294, 326, 327, 34..."
11,11,1,12,18,1939,Germany,Wilhelmshaven,Germany,53.53234,8.106872,"[44, 78, 102, 172, 185, 243, 294, 326, 327, 34..."


## Query the database
Now that we can link the weather stations that were in the vicinity of a bombing mission, we can use the StationID to extract the weather information reported by each of those stations.  Since most (if not all) bombing missions were in range of several weather stations, it makes sense to use the average value of each parameter measured by the stations as our metric for evaluating the weather on that day for that specific attack.

To do that, we'll need to gather the attributes from the weather database that corresponds to the StationIDs collected on a given day of an attack.

In order to query the database, we'll need pass in a string that contains the SQL statement that we want to execute.  Then, convert degrees to farenheight and return the resulting dataframe.  To complicate matters, some values have -999.  These will have to be omitted so the result isn't polluted with extreme values when averaged.

In [279]:
# Define the query and load the data.  MaxWindGust and MeanWindSpeed values are all -999.0.  Remove from query
def GetWeather(query):
    query_job = client.query(query)
    weather_query = query_job.to_dataframe()

# Convert Temps to Farenheit
    try:
        weather_query['MaxTemp'] = weather_query['MaxTemp'].apply(lambda x: float((x * 9/5) + 32))
        weather_query['MinTemp'] = weather_query['MinTemp'].apply(lambda x: float((x * 9/5) + 32))
        weather_query['MeanTemp'] = weather_query['MeanTemp'].apply(lambda x: float((x * 9/5) + 32))
        weather_query['MinAirTemp'] = weather_query['MinAirTemp'].apply(lambda x: float((x * 9/5) + 32))
        weather_query.loc[weather_query['MaxTemp']<=-999.0, ['MaxTemp']]= -999.0
        weather_query.loc[weather_query['MinTemp']<=-999.0, ['MinTemp']]= -999.0
        weather_query.loc[weather_query['MeanTemp']<=-999.0, ['MeanTemp']]= -999.0
        weather_query.loc[weather_query['MinAirTemp']<=-999.0, ['MinAirTemp']]= -999.0
    except:
        print(query + " was unable to process because no weather stations were in range.")
    
# Make a new dataframe that will just be 1 row long.
    weather= weather_query.iloc[0,:]
# Then, we'll need to filter out anywhere that a value equals -999.
    def BadValues(values):
        if values==-999:
            return False
        else:
            return True
# Average the resulting table, but only where values are not -999.  Take the median of PrecipForm because it's a factor.
# If this fails, it's because there are no values that do not equal -999.
    try:
        weather["MinTemp"]= sum(filter(BadValues, weather_query["MinTemp"])) / len(list(filter(BadValues, weather_query["MinTemp"])))
    except:
        print("This one failed.  MinTemp had no non -999 values")
    try:
        weather["MaxTemp"]= sum(filter(BadValues, weather_query["MaxTemp"])) / len(list(filter(BadValues, weather_query["MaxTemp"])))
    except:
        print("This one failed.  MaxTemp had no non -999 values")
    try:
        weather["MeanTemp"]= sum(filter(BadValues, weather_query["MeanTemp"])) / len(list(filter(BadValues, weather_query["MeanTemp"])))
    except:
        print("This one failed.  MeanTemp had no non -999 values")
    try:
        weather["MinAirTemp"]= sum(filter(BadValues, weather_query["MinAirTemp"])) / len(list(filter(BadValues, weather_query["MinAirTemp"])))
    except:
        print("This one failed.  MinAirTemp had no non -999 values")
    try:
        weather["SunDuration"]= sum(filter(BadValues, weather_query["SunDuration"])) / len(list(filter(BadValues, weather_query["SunDuration"])))
    except:
        print("This one failed.  SunDuration had no non -999 values")
    try:
        weather["MeanCloudCover"]= sum(filter(BadValues, weather_query["MeanCloudCover"])) / len(list(filter(BadValues, weather_query["MeanCloudCover"])))
    except:
        print("This one failed.  MeanCloudCover had no non -999 values")
    try:
        weather["MeanCloudVapor"]= sum(filter(BadValues, weather_query["MeanCloudVapor"])) / len(list(filter(BadValues, weather_query["MeanCloudVapor"])))
    except:
        print("This one failed.  MeanCloudVapor had no non -999 values")
    try:
        weather["MeanRelHumid"]= sum(filter(BadValues, weather_query["MeanRelHumid"])) / len(list(filter(BadValues, weather_query["MeanRelHumid"])))
    except:
        print("This one failed.  MeanRelHumid had no non -999 values")
    try:
        weather["PrecipHeight"]= sum(filter(BadValues, weather_query["PrecipHeight"])) / len(list(filter(BadValues, weather_query["PrecipHeight"])))
    except:    
        print("This one failed.  PrecipHeight had no non -999 values")
    try: 
        weather["PrecipForm"]= statistics.median(filter(BadValues, weather_query["PrecipForm"]))
    except:
        print("This one failed.  PrecipForm had no non -999 values")
    try:
        weather["MeanPressure"]= sum(filter(BadValues, weather_query["MeanPressure"])) / len(list(filter(BadValues, weather_query["MeanPressure"])))
    except:
        print("This one failed.  MeanPressure had no non -999 values")
    try:
        weather["SnowDepth"]= sum(filter(BadValues, weather_query["SnowDepth"])) / len(list(filter(BadValues, weather_query["SnowDepth"])))
    except:
        print("This one failed.  SnowDepth had no non -999 values")
    return(weather)

In [280]:
bombing_stations["MinTemp"]= ""
bombing_stations["MaxTemp"]= ""
bombing_stations["MeanTemp"]= ""
bombing_stations["MinAirTemp"]= ""
bombing_stations["SunDuration"]= ""
bombing_stations["MeanCloudCover"]= ""
bombing_stations["MeanCloudVapor"]= ""
bombing_stations["MeanRelHumid"]= ""
bombing_stations["PrecipHeight"]= ""
bombing_stations["PrecipForm"]= ""
bombing_stations["MeanPressure"]= ""
bombing_stations["SnowDepth"]= ""
#main = """SELECT AVG(MinTemp) MinTemp, AVG(MaxTemp) MaxTemp, AVG(MeanTemp) MeanTemp, AVG(MinAirTemp) MinAirTemp,
#    AVG(SunDuration) SunDuration, AVG(MeanCloudCover) MeanCloudCover, AVG(MeanCloudVapor) MeanCloudVapor,
#    AVG(MeanRelHumid) MeanRelHumid, AVG(PrecipHeight) PrecipHeight, AVG(PrecipForm) PrecipForm,
#    AVG(MeanPressure) MeanPressure, AVG(SnowDepth) SnowDepth
#    FROM `ml-jth.germany.weather`
#    WHERE date = DATE("""
main = """SELECT MinTemp MinTemp, MaxTemp MaxTemp, MeanTemp MeanTemp, MinAirTemp MinAirTemp,
    SunDuration SunDuration, MeanCloudCover MeanCloudCover, MeanCloudVapor MeanCloudVapor,
    MeanRelHumid MeanRelHumid, PrecipHeight PrecipHeight, PrecipForm PrecipForm,
    MeanPressure MeanPressure, SnowDepth SnowDepth
    FROM `ml-jth.germany.weather`
    WHERE date = DATE("""
print("Collecting data...")
i= 0
for index, row in bombing_stations.iterrows():
    date= str(row.Year) + "-" + str(row.Month) + "-" + str(row.Day)
    print(i, index, date)
    query =  main + '"{}"'.format(date) + ") AND StationID IN " + "("+ str(row.StationIDs)[1:-1] + ")"
    weather = GetWeather(query)
    bombing_stations.iloc[i,11] = weather["MinTemp"]
    bombing_stations.iloc[i,12] = weather["MaxTemp"]
    bombing_stations.iloc[i,13] = weather["MeanTemp"]
    bombing_stations.iloc[i,14] = weather["MinAirTemp"]
    bombing_stations.iloc[i,15] = weather["SunDuration"]
    bombing_stations.iloc[i,16] = weather["MeanCloudCover"]
    bombing_stations.iloc[i,17] = weather["MeanCloudVapor"]
    bombing_stations.iloc[i,18] = weather["MeanRelHumid"]
    bombing_stations.iloc[i,19] = weather["PrecipHeight"]
    bombing_stations.iloc[i,20] = weather["PrecipForm"]
    bombing_stations.iloc[i,21] = weather["MeanPressure"]
    bombing_stations.iloc[i,22] = weather["SnowDepth"]
    i += 1
print("Complete")
    

0 3 1939-9-3
1 4 1939-9-3
2 5 1939-9-4
3 6 1939-9-4
4 11 1939-12-18
5 18 1940-5-15
6 19 1940-5-19
7 24 1940-6-7
8 30 1940-8-25
9 37 1941-1-21
10 39 1941-3-31
11 43 1941-8-8
12 46 1941-9-7
13 47 1941-11-7
14 48 1941-12-7
15 51 1942-3-8
16 52 1942-3-13
17 53 1942-3-25
18 54 1942-3-28
19 55 1942-4-8
20 56 1942-4-17
21 57 1942-4-23
22 58 1942-4-24
23 59 1942-5-30
24 61 1942-6-25
25 70 1942-9-2
26 74 1942-12-22
27 75 1943-1-27
28 76 1943-3-5
29 77 1943-4-13
30 80 1943-5-17
31 81 1943-6-11
32 82 1943-6-13
33 83 1943-6-26
34 84 1943-6-20
35 85 1943-7-19
36 86 1943-7-24
37 91 1943-8-17
38 92 1943-8-18
39 96 1943-10-10
40 97 1943-10-14
41 98 1943-11-1
42 101 1943-11-3
43 102 1943-11-18
44 103 1943-11-22
45 109 1944-2-19
46 110 1944-2-20
47 111 1944-3-6
48 112 1944-3-1
49 122 1944-7-23
50 124 1944-7-26
51 125 1944-7-28
52 126 1944-8-27
53 133 1945-2-3
54 134 1945-2-13
55 135 1945-3-12
56 136 1945-3-14
57 137 1945-2-1
58 138 1945-3-17
59 139 1945-3-18
60 140 1945-3-22
61 143 1945-5-2
62 144 1945-

In [282]:
from IPython.display import display

pd.options.display.max_columns = None
pd.options.display.max_rows = None
bombing_stations.head()

,Unnamed: 0,Success,Month,Day,Year,Country,DefCity,DefCountry,Latitude,Longitude,StationIDs,MinTemp,MaxTemp,MeanTemp,MinAirTemp,SunDuration,MeanCloudCover,MeanCloudVapor,MeanRelHumid,PrecipHeight,PrecipForm,MeanPressure,SnowDepth
3,3,0,9,3,1939,United Kingdom,Wilhelmshaven,Germany,53.53234,8.106872,"[44, 78, 102, 172, 185, 243, 294, 326, 327, 34...",59.12,78.02,67.868,55.295,7.75714,4.5,17.1533,73.8,1.43333,0,1011.78,0
4,4,0,9,3,1939,United Kingdom,Wilhelmshaven,Germany,53.53234,8.106872,"[44, 78, 102, 172, 185, 243, 294, 326, 327, 34...",59.12,78.02,67.868,55.295,7.75714,4.5,17.1533,73.8,1.43333,0,1011.78,0
5,5,0,9,4,1939,United Kingdom,Wilhelmshaven,Germany,53.53234,8.106872,"[44, 78, 102, 172, 185, 243, 294, 326, 327, 34...",58.568,69.836,63.512,56.735,1.95714,6.57333,15.94,79.2667,8.81333,1,1010.03,0
6,6,0,9,4,1939,United Kingdom,Wilhelmshaven,Germany,53.53234,8.106872,"[44, 78, 102, 172, 185, 243, 294, 326, 327, 34...",58.568,69.836,63.512,56.735,1.95714,6.57333,15.94,79.2667,8.81333,1,1010.03,0
11,11,1,12,18,1939,Germany,Wilhelmshaven,Germany,53.53234,8.106872,"[44, 78, 102, 172, 185, 243, 294, 326, 327, 34...",16.22,28.424,22.676,13.7923,2.775,2.18,3.52,83.8,0,0,1021.32,0.0666667


The resulting dataframe contains the weather parameters averaged over all of the stations that were in range of each attack.  While the column "StationIDs" represents the stations that were in range of the attack, it does **not** reflect which stations actually had data on that day.

All that is left to do now is convert the dataframe to a .csv and upload it for future use.

In [283]:
# Convert the pandas dataframe to a .csv
bombing_stations.to_csv("Bombing_Weather_Data.csv")